In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("HELK Reader") \
    .master("spark://helk-spark-master:7077") \
    .enableHiveSupport() \
    .getOrCreate()

In [3]:
es_reader = (spark.read
    .format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.read.field.as.array.include", "tags")
    .option("es.nodes","helk-elasticsearch:9200")
    .option("es.net.http.auth.user","elastic")
)

In [4]:
sysmon_df = es_reader.load("logs-endpoint-winevent-sysmon-*/")

In [5]:
sysmon_df.createOrReplaceTempView("sysmon_events")

In [36]:
DLL_Injection_EID_8_10 = spark.sql(
'''
SELECT 
    a.host_name,
    a.process_name,
    a.process_granted_access,
    a.process_id, 
    a.process_target_name,
    c.thread_new_id
FROM sysmon_events c
JOIN sysmon_events a
    ON a.process_guid = c.process_guid
    AND a.event_id = 10
    AND (a.process_granted_access & 40) == 40 -- 40 is decimal for 0x0028. The minimal privileges you need to access process handle
WHERE
    c.event_id = 8
    AND c.thread_start_function = "LoadLibraryA"
'''
).show(100,False)

+---------------+--------------+----------------------+----------+-------------------+-------------+
|host_name      |process_name  |process_granted_access|process_id|process_target_name|thread_new_id|
+---------------+--------------+----------------------+----------+-------------------+-------------+
|hr001.shire.com|powershell.exe|2047999               |5452      |notepad.exe        |7940         |
+---------------+--------------+----------------------+----------+-------------------+-------------+



In [38]:
DLL_Injection_EID_7_8_10 = spark.sql(
'''
SELECT 
    a.host_name,
    a.process_name,
    a.process_granted_access,
    a.process_id, 
    a.process_target_name,
    c.thread_new_id
FROM sysmon_events b
JOIN sysmon_events a
    ON a.process_target_guid = b.process_guid
    AND a.event_id = 10
    AND (a.process_granted_access & 5184) == 5184 -- 40 is decimal for 0x0028. The minimal privileges you need to access process handle
JOIN sysmon_events c
    ON c.process_guid = a.process_guid
    AND c.event_id = 8
    AND c.thread_start_function = "LoadLibraryA"
WHERE
     b.event_id = 7
'''
).show(1,False)

+---------------+--------------+----------------------+----------+-------------------+-------------+
|host_name      |process_name  |process_granted_access|process_id|process_target_name|thread_new_id|
+---------------+--------------+----------------------+----------+-------------------+-------------+
|hr001.shire.com|powershell.exe|2047999               |5452      |notepad.exe        |7940         |
+---------------+--------------+----------------------+----------+-------------------+-------------+
only showing top 1 row

